# Module 4: A2A (Agent-to-Agent) Protocol

**Duration:** ~40 minutes

**Learning Objectives:**
- Understand the A2A protocol architecture and use cases
- Create Agent Cards to describe agent capabilities
- Expose ADK agents as A2A servers
- Consume remote A2A agents with RemoteA2aAgent
- Build multi-agent systems that communicate via A2A

---

## 4.1 Understanding A2A

### What is A2A?

The **Agent2Agent (A2A) Protocol** is an open standard developed by Google that enables AI agents to communicate and collaborate with each other, regardless of:
- The framework they're built with (ADK, LangChain, CrewAI, etc.)
- The organization that built them
- The server they're running on

### A2A vs MCP

```
┌─────────────────────────────────────────────────────────────────┐
│                   A2A vs MCP Comparison                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   MCP (Model Context Protocol)      A2A (Agent-to-Agent)        │
│   ─────────────────────────────     ─────────────────────       │
│                                                                  │
│   ┌───────┐        ┌──────┐        ┌───────┐      ┌───────┐    │
│   │ Agent │───────►│ Tool │        │ Agent │◄────►│ Agent │    │
│   └───────┘        └──────┘        └───────┘      └───────┘    │
│                                                                  │
│   • Agent calls tool               • Agents collaborate          │
│   • Tool executes function         • Both can reason             │
│   • Simple request/response        • Multi-turn dialogue         │
│   • Tool has no autonomy           • Agents are autonomous       │
│                                                                  │
│   Example: Agent queries database  Example: Travel agent asks   │
│                                    hotel agent for availability │
└─────────────────────────────────────────────────────────────────┘
```

### When to Use A2A

| Use A2A When | Example |
|--------------|--------|
| Agents need to **collaborate** | Planning agent + Booking agent |
| Agents are **owned by different teams** | Internal agent + Vendor agent |
| Agents are **built on different frameworks** | ADK agent + LangGraph agent |
| You need **multi-turn interactions** | Negotiation, clarification |
| Agents should **remain opaque** | No shared memory/state |

## 4.2 A2A Architecture

### Core Components

```
┌─────────────────────────────────────────────────────────────────┐
│                      A2A Architecture                            │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌─────────────────────┐          ┌─────────────────────┐       │
│  │   A2A CLIENT        │          │   A2A SERVER        │       │
│  │   (Consumer)        │          │   (Provider)        │       │
│  │                     │          │                     │       │
│  │ ┌─────────────────┐ │          │ ┌─────────────────┐ │       │
│  │ │ RemoteA2aAgent  │ │◄────────►│ │   Your Agent    │ │       │
│  │ └─────────────────┘ │  JSON-   │ └─────────────────┘ │       │
│  │         │           │   RPC    │         │           │       │
│  │         ▼           │          │         ▼           │       │
│  │ ┌─────────────────┐ │          │ ┌─────────────────┐ │       │
│  │ │  Agent Card     │ │◄─────────│ │  Agent Card     │ │       │
│  │ │  (Discovery)    │ │  HTTP    │ │  (Published)    │ │       │
│  │ └─────────────────┘ │          │ └─────────────────┘ │       │
│  └─────────────────────┘          └─────────────────────┘       │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### Key Concepts

1. **Agent Card**: A JSON file describing an agent's capabilities
   - Name, description, version
   - Skills and capabilities
   - Endpoint URL
   - Authentication requirements

2. **Task**: A unit of work sent between agents
   - Has a lifecycle: submitted → working → completed/failed
   - Can require multiple turns (input-required state)

3. **Message**: Communication within a task
   - Contains text, files, or structured data
   - Can be from user or agent role

4. **Artifact**: Output produced by an agent
   - Documents, images, structured data

In [ ]:
# Setup
import os
from getpass import getpass

if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass("Enter your Gemini API Key: ")

# Install A2A-related packages
!pip install -q google-adk uvicorn httpx pydantic

print("✅ Setup complete!")

## 4.3 Agent Cards

An **Agent Card** is like a business card for your agent. It tells other agents:
- What your agent can do (skills)
- How to contact it (endpoint)
- What data formats it supports (modalities)

Agent Cards are typically served at `/.well-known/agent.json`

In [ ]:
# Example Agent Card structure
import json

example_agent_card = {
    "name": "Currency Exchange Agent",
    "description": "An agent that provides real-time currency exchange rates and conversions",
    "version": "1.0.0",
    "url": "https://my-agent.example.com",
    "capabilities": {
        "streaming": True,
        "pushNotifications": False
    },
    "skills": [
        {
            "id": "currency_conversion",
            "name": "Currency Conversion",
            "description": "Convert amounts between different currencies using real-time rates",
            "examples": [
                "Convert 100 USD to EUR",
                "What's the current exchange rate for GBP to JPY?"
            ]
        },
        {
            "id": "rate_history",
            "name": "Rate History",
            "description": "Get historical exchange rates for currency pairs",
            "examples": [
                "Show me USD/EUR rate for the past week"
            ]
        }
    ],
    "defaultInputModes": ["text"],
    "defaultOutputModes": ["text"],
    "authentication": {
        "schemes": ["none"]
    }
}

print("📇 Example Agent Card:")
print("=" * 60)
print(json.dumps(example_agent_card, indent=2))

In [ ]:
# A2A SDK provides Pydantic models for Agent Cards
from pydantic import BaseModel
from typing import List, Optional, Dict, Any

class AgentSkill(BaseModel):
    """Describes a specific capability of the agent"""
    id: str
    name: str
    description: str
    examples: Optional[List[str]] = None
    tags: Optional[List[str]] = None

class AgentCapabilities(BaseModel):
    """Technical capabilities of the agent"""
    streaming: bool = False
    pushNotifications: bool = False

class AgentCard(BaseModel):
    """Complete Agent Card specification"""
    name: str
    description: str
    version: str
    url: str
    skills: List[AgentSkill]
    capabilities: AgentCapabilities = AgentCapabilities()
    defaultInputModes: List[str] = ["text"]
    defaultOutputModes: List[str] = ["text"]

# Create an agent card using Pydantic
travel_agent_card = AgentCard(
    name="Travel Planner Agent",
    description="An AI agent that helps plan trips, find flights, and book accommodations",
    version="1.0.0",
    url="http://localhost:8001",
    skills=[
        AgentSkill(
            id="flight_search",
            name="Flight Search",
            description="Search for flights between cities",
            examples=["Find flights from NYC to London"]
        ),
        AgentSkill(
            id="hotel_search",
            name="Hotel Search",
            description="Find hotels in a city",
            examples=["Find hotels in Paris for next weekend"]
        )
    ],
    capabilities=AgentCapabilities(streaming=True)
)

print("📇 Travel Agent Card (Pydantic):")
print("=" * 60)
print(travel_agent_card.model_dump_json(indent=2))

## 4.4 Exposing an Agent via A2A

There are two main ways to expose an ADK agent via A2A:

### Method 1: Using `to_a2a()` (Recommended)
- Automatically generates Agent Card from your agent
- Simpler setup
- Good for quick prototyping

### Method 2: Using `adk api_server --a2a`
- Requires manual Agent Card creation
- More control over the card
- Works with `adk web` for debugging

In [ ]:
# Method 1: Using to_a2a() - Code Template

to_a2a_template = '''
# remote_agent/server.py
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from google.adk.a2a import to_a2a
import uvicorn

# Define your agent
def check_prime(number: int) -> dict:
    """Check if a number is prime."""
    if number < 2:
        return {"number": number, "is_prime": False}
    for i in range(2, int(number ** 0.5) + 1):
        if number % i == 0:
            return {"number": number, "is_prime": False}
    return {"number": number, "is_prime": True}

prime_agent = Agent(
    name="prime_checker",
    model="gemini-2.0-flash",
    description="An agent that checks if numbers are prime",
    instruction="You help users check if numbers are prime. Use the check_prime tool.",
    tools=[FunctionTool(check_prime)]
)

# Convert to A2A server - this auto-generates the Agent Card!
a2a_app = to_a2a(prime_agent)

if __name__ == "__main__":
    # Run the A2A server
    uvicorn.run(a2a_app, host="0.0.0.0", port=8001)
'''

print("📝 to_a2a() Server Template:")
print("=" * 60)
print(to_a2a_template)

In [ ]:
# Method 2: Manual Agent Card - Code Template

manual_agent_card_template = '''
# my_agent/agent.json
{
    "name": "Prime Checker Agent",
    "description": "An agent that determines if numbers are prime",
    "version": "1.0.0",
    "url": "http://localhost:8001",
    "skills": [
        {
            "id": "check_prime",
            "name": "Prime Number Check",
            "description": "Determines whether a given number is prime",
            "examples": [
                "Is 17 a prime number?",
                "Check if 100 is prime"
            ]
        }
    ],
    "capabilities": {
        "streaming": false
    },
    "defaultInputModes": ["text"],
    "defaultOutputModes": ["text"]
}
'''

# Then run with:
# adk api_server --a2a --port 8001 my_agent/

print("📝 Manual Agent Card Template (agent.json):")
print("=" * 60)
print(manual_agent_card_template)

## 4.5 Consuming Remote A2A Agents

To use a remote A2A agent, you use `RemoteA2aAgent` in ADK. This acts as a proxy that handles all the A2A protocol communication.

In [ ]:
# Consuming an A2A agent - Code Template

consume_a2a_template = '''
from google.adk.agents import Agent, RemoteA2aAgent
from google.adk.runners import InMemoryRunner

# Reference the remote agent by its Agent Card URL
remote_prime_agent = RemoteA2aAgent(
    name="remote_prime_checker",
    description="Remote agent that checks if numbers are prime",
    agent_card="http://localhost:8001/.well-known/agent.json"
)

# Create a root agent that uses the remote agent
root_agent = Agent(
    name="math_assistant",
    model="gemini-2.0-flash",
    description="A math assistant that can delegate to specialists",
    instruction="""
    You are a math assistant. When asked about prime numbers,
    delegate to the remote_prime_checker agent.
    """,
    sub_agents=[remote_prime_agent]  # Remote agent as sub-agent!
)

# Run the agent
async def main():
    runner = InMemoryRunner(agent=root_agent)
    session = await runner.session_service.create_session(
        app_name=root_agent.name,
        user_id="user"
    )
    
    response = await runner.run(
        user_id="user",
        session_id=session.id,
        new_message="Is 97 a prime number?"
    )
    
    async for event in response:
        if hasattr(event, "content"):
            print(event.content)
'''

print("📝 RemoteA2aAgent Consumer Template:")
print("=" * 60)
print(consume_a2a_template)

## 4.6 Simulating A2A in Colab

Since we can't run multiple servers in Colab easily, let's simulate the A2A pattern by building agents that would work over A2A.

In [ ]:
# Simulated A2A scenario: Multi-agent travel planning

from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from google.adk.runners import InMemoryRunner
from typing import Dict, Any, List
import random

# Agent 1: Flight Specialist (would be a remote A2A agent)
def search_flights(
    origin: str,
    destination: str,
    date: str
) -> Dict[str, Any]:
    """
    Search for available flights.
    
    Args:
        origin: Departure city or airport code
        destination: Arrival city or airport code
        date: Travel date (YYYY-MM-DD)
    
    Returns:
        List of available flights
    """
    flights = []
    for i in range(3):
        flights.append({
            "flight_number": f"FL{random.randint(100,999)}",
            "departure": f"{6 + i*4:02d}:00",
            "arrival": f"{9 + i*4:02d}:30",
            "price": random.randint(200, 600),
            "airline": random.choice(["United", "Delta", "American"])
        })
    return {
        "status": "success",
        "origin": origin,
        "destination": destination,
        "date": date,
        "flights": flights
    }

flight_agent = Agent(
    name="flight_specialist",
    model="gemini-2.0-flash",
    description="Specialist for finding and booking flights. Delegate flight-related queries here.",
    instruction="You are a flight booking specialist. Search for flights and present options clearly.",
    tools=[FunctionTool(search_flights)]
)

print("✅ Flight Specialist agent created")

In [ ]:
# Agent 2: Hotel Specialist (would be another remote A2A agent)

def search_hotels(
    city: str,
    check_in: str,
    check_out: str,
    guests: int = 2
) -> Dict[str, Any]:
    """
    Search for available hotels.
    
    Args:
        city: City to search for hotels
        check_in: Check-in date (YYYY-MM-DD)
        check_out: Check-out date (YYYY-MM-DD)
        guests: Number of guests
    
    Returns:
        List of available hotels
    """
    hotel_names = ["Grand Plaza", "City Inn", "Comfort Suites", "Luxury Resort", "Budget Stay"]
    hotels = []
    for i in range(3):
        hotels.append({
            "hotel_name": random.choice(hotel_names) + f" {city}",
            "rating": round(random.uniform(3.5, 5.0), 1),
            "price_per_night": random.randint(80, 350),
            "amenities": random.sample(["WiFi", "Pool", "Gym", "Breakfast", "Parking"], 3)
        })
    return {
        "status": "success",
        "city": city,
        "check_in": check_in,
        "check_out": check_out,
        "hotels": hotels
    }

hotel_agent = Agent(
    name="hotel_specialist",
    model="gemini-2.0-flash",
    description="Specialist for finding hotels and accommodations. Delegate hotel queries here.",
    instruction="You are a hotel booking specialist. Search for hotels and highlight key features.",
    tools=[FunctionTool(search_hotels)]
)

print("✅ Hotel Specialist agent created")

In [ ]:
# Agent 3: Weather Specialist

def get_weather_forecast(
    city: str,
    days: int = 5
) -> Dict[str, Any]:
    """
    Get weather forecast for a city.
    
    Args:
        city: City to get forecast for
        days: Number of days to forecast
    
    Returns:
        Weather forecast
    """
    conditions = ["Sunny", "Partly Cloudy", "Cloudy", "Rainy", "Thunderstorms"]
    forecast = []
    for i in range(min(days, 7)):
        forecast.append({
            "day": f"Day {i+1}",
            "condition": random.choice(conditions),
            "high": random.randint(65, 85),
            "low": random.randint(45, 65)
        })
    return {
        "status": "success",
        "city": city,
        "forecast": forecast
    }

weather_agent = Agent(
    name="weather_specialist",
    model="gemini-2.0-flash",
    description="Specialist for weather forecasts. Delegate weather queries here.",
    instruction="You are a weather specialist. Provide forecasts and packing recommendations.",
    tools=[FunctionTool(get_weather_forecast)]
)

print("✅ Weather Specialist agent created")

In [ ]:
# Orchestrator Agent (would use RemoteA2aAgent in production)

travel_orchestrator = Agent(
    name="travel_planner",
    model="gemini-2.0-flash",
    description="A comprehensive travel planning assistant",
    instruction="""
    You are a travel planning assistant that coordinates with specialist agents.
    
    When helping users plan trips:
    1. For flight queries → delegate to flight_specialist
    2. For hotel queries → delegate to hotel_specialist  
    3. For weather queries → delegate to weather_specialist
    
    For comprehensive trip planning, gather information from all relevant specialists
    and synthesize it into a coherent travel plan.
    
    Always be helpful and provide detailed recommendations.
    """,
    sub_agents=[flight_agent, hotel_agent, weather_agent]
)

print(f"✅ Travel Orchestrator created with {len(travel_orchestrator.sub_agents)} sub-agents")
print(f"   Sub-agents: {[a.name for a in travel_orchestrator.sub_agents]}")

In [ ]:
# Helper function
async def chat_with_agent(agent: Agent, message: str) -> str:
    runner = InMemoryRunner(agent=agent)
    session = await runner.session_service.create_session(
        app_name=agent.name,
        user_id="workshop_user"
    )
    
    response_text = ""
    response = await runner.run(
        user_id="workshop_user",
        session_id=session.id,
        new_message=message
    )
    
    async for event in response:
        if hasattr(event, 'content') and event.content:
            for part in event.content.parts:
                if hasattr(part, 'text'):
                    response_text += part.text
    
    return response_text

In [ ]:
# Test the multi-agent system

response = await chat_with_agent(
    travel_orchestrator,
    """I'm planning a trip from New York to Paris next month (March 15-22, 2025).
    Can you help me find flights, hotels, and tell me what the weather might be like?"""
)

print("✈️🏨🌤️ Multi-Agent Travel Planning Test")
print("=" * 60)
print(response)

## 4.7 A2A Task Lifecycle

Understanding the task lifecycle is important for building robust A2A agents:

```
┌─────────────────────────────────────────────────────────────────┐
│                     A2A Task Lifecycle                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   ┌──────────┐                                                  │
│   │ SUBMITTED│ ────► Task received by server                    │
│   └────┬─────┘                                                  │
│        │                                                         │
│        ▼                                                         │
│   ┌──────────┐                                                  │
│   │ WORKING  │ ────► Agent processing the task                  │
│   └────┬─────┘                                                  │
│        │                                                         │
│        ├─────────────────┬────────────────┐                     │
│        │                 │                │                     │
│        ▼                 ▼                ▼                     │
│   ┌──────────┐    ┌───────────┐    ┌──────────┐                │
│   │COMPLETED │    │INPUT_REQ'D│    │  FAILED  │                │
│   └──────────┘    └─────┬─────┘    └──────────┘                │
│                         │                                        │
│                         │ (client sends more input)             │
│                         ▼                                        │
│                    ┌──────────┐                                 │
│                    │ WORKING  │ ────► Process continues         │
│                    └──────────┘                                 │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Example: Agent that requires multiple turns (input_required)

interactive_booking_template = '''
# This pattern shows how an A2A agent can request additional input

from google.adk.agents import Agent
from google.adk.tools import FunctionTool

def initiate_booking(flight_number: str, passenger_name: str) -> dict:
    """
    Start a flight booking. Returns confirmation request.
    """
    return {
        "status": "confirmation_required",
        "message": f"Please confirm booking for {passenger_name} on flight {flight_number}",
        "details": {
            "flight": flight_number,
            "passenger": passenger_name,
            "price": 350.00
        },
        "required_action": "Reply with CONFIRM to complete booking"
    }

def confirm_booking(confirmation_code: str) -> dict:
    """
    Confirm a pending booking.
    """
    if confirmation_code.upper() == "CONFIRM":
        return {
            "status": "success",
            "confirmation_number": "ABC123",
            "message": "Booking confirmed!"
        }
    return {
        "status": "cancelled",
        "message": "Booking cancelled."
    }

# Agent that handles multi-turn booking flow
booking_agent = Agent(
    name="booking_agent",
    model="gemini-2.0-flash",
    instruction="""
    Handle flight bookings with confirmation.
    1. When user wants to book, use initiate_booking
    2. Wait for user confirmation
    3. Use confirm_booking to complete
    """,
    tools=[FunctionTool(initiate_booking), FunctionTool(confirm_booking)]
)
'''

print("📝 Multi-Turn A2A Agent Pattern:")
print("=" * 60)
print(interactive_booking_template)

## 4.8 A2A Production Patterns

### Deployment Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                 A2A Production Deployment                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   ┌─────────────────────────────────────────────────────────┐   │
│   │                    Load Balancer                         │   │
│   └──────────────────────────┬──────────────────────────────┘   │
│                              │                                   │
│      ┌───────────────────────┼───────────────────────┐          │
│      │                       │                       │          │
│  ┌───▼───┐              ┌────▼────┐             ┌───▼───┐       │
│  │Cloud  │              │ Agent   │             │ Vertex│       │
│  │Run    │              │ Engine  │             │  AI   │       │
│  │       │              │         │             │       │       │
│  │A2A    │              │ A2A     │             │ A2A   │       │
│  │Agent 1│              │ Agent 2 │             │Agent 3│       │
│  └───────┘              └─────────┘             └───────┘       │
│                                                                  │
│   ┌─────────────────────────────────────────────────────────┐   │
│   │              Shared Services                             │   │
│   │  • Agent Registry (Agent Card discovery)                 │   │
│   │  • Authentication (OAuth, API Keys)                      │   │
│   │  • Observability (Tracing, Logging)                      │   │
│   └─────────────────────────────────────────────────────────┘   │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### Security Considerations

1. **Authentication**: Use OAuth or API keys
2. **Authorization**: Validate agent permissions
3. **Rate Limiting**: Protect against abuse
4. **Input Validation**: Sanitize all inputs
5. **Audit Logging**: Track all agent interactions

In [ ]:
# A2A with Authentication - Code Pattern

authenticated_a2a = '''
# agent_card.json with authentication
{
    "name": "Secure Finance Agent",
    "description": "Agent for financial operations",
    "version": "1.0.0",
    "url": "https://finance-agent.example.com",
    "authentication": {
        "schemes": ["oauth2", "bearer"],
        "oauth2": {
            "authorizationUrl": "https://auth.example.com/authorize",
            "tokenUrl": "https://auth.example.com/token",
            "scopes": {
                "read:accounts": "Read account information",
                "write:transactions": "Execute transactions"
            }
        }
    },
    "skills": [...]
}
'''

print("🔐 Authenticated A2A Agent Card:")
print("=" * 60)
print(authenticated_a2a)

## 4.9 Exercise: Design an A2A System

Design a multi-agent system for an e-commerce platform:

1. **Product Agent**: Search and recommend products
2. **Order Agent**: Handle orders and returns
3. **Support Agent**: Answer customer questions
4. **Orchestrator**: Route requests to appropriate agents

In [ ]:
# Exercise: Create Agent Cards for the e-commerce system

# TODO: Define Agent Card for Product Agent
product_agent_card = {
    "name": "Product Catalog Agent",
    "description": "...",  # Fill in
    "skills": [
        # Add skills
    ]
}

# TODO: Define Agent Card for Order Agent
order_agent_card = {
    # Fill in
}

# TODO: Define Agent Card for Support Agent
support_agent_card = {
    # Fill in
}

---

## Summary

In this module, you learned:

1. **A2A Protocol**: Enables agent-to-agent collaboration
2. **Agent Cards**: Describe agent capabilities for discovery
3. **to_a2a()**: Quick way to expose ADK agents via A2A
4. **RemoteA2aAgent**: Consume remote A2A agents in your code
5. **Task Lifecycle**: Understanding multi-turn interactions
6. **Production Patterns**: Authentication, security, deployment

### Key Takeaways

- A2A complements MCP: MCP for tools, A2A for agents
- Agent Cards are essential for discovery and interoperability
- ADK makes it easy to both expose and consume A2A agents
- Multi-agent systems enable complex workflows

### Next Steps

Continue to **Module 5: Combining A2A and MCP** to learn how these protocols work together.

---

### Resources

- [A2A Protocol Specification](https://a2a-protocol.org/)
- [ADK A2A Documentation](https://google.github.io/adk-docs/a2a/)
- [A2A Python SDK](https://github.com/a2aproject/a2a-python)
- [Google Cloud A2A Blog](https://cloud.google.com/blog/products/ai-machine-learning/agent2agent-protocol-is-getting-an-upgrade)